# Using Table and Data 

### Importing Libraries

In [1]:
import sqlite3

In [2]:
# create connection  and cursor object
connection = sqlite3.connect("pizza_runner_database")
cursor = connection.cursor()

In [3]:
# see the tables in database
query ="select name from sqlite_master  where type='table';"
cursor.execute(query)
result = cursor.fetchall()
print('Tables: {}'.format(result))

Tables: [('runners',), ('customer_orders',), ('runner_orders',), ('pizza_names',), ('pizza_recipes',), ('pizza_toppings',)]


### Data Cleaning and Transformation

In this we will directly create view of corrected data table. Overall process is discussed in clean_data.ipynb file

In [4]:
#for customer_orders table
cursor.execute("DROP View IF EXISTS c_view;")
query= """
create view c_view as 

select order_id, customer_id, pizza_id,
case
    when exclusions = 'null' or TRIM(exclusions) ="" then NULL
    else exclusions
end as exclusions,
case
    when extras = 'null' or TRIM(extras) ="" then NULL
    else extras
end as extras,
order_time

from customer_orders;


"""
cursor.execute(query)


# for runner_orders table
cursor.execute("DROP View IF EXISTS r_view;")
query= """
create view r_view as 

select order_id, runner_id,
case
    when pickup_time  = 'null'  then NULL
    else datetime(pickup_time)
end as pickup_time,
CASE
     WHEN distance = 'null' or TRIM(distance)= "" THEN NULL
     WHEN distance LIKE '%km' THEN cast(REPLACE(distance, 'km', '') as real)
     ELSE cast(distance as real) 
   END AS distance,
CASE
     WHEN duration = 'null' or TRIM(duration)="" THEN NULL
     WHEN duration LIKE '%mins' THEN cast(REPLACE(duration, 'mins', '') as integer)
     WHEN duration LIKE '%minute' THEN cast(REPLACE(duration, 'minute', '') as integer)
     WHEN duration LIKE '%minutes' THEN cast(REPLACE(duration, 'minutes', '') as integer)
     ELSE cast(duration as integer)
   END AS duration,
case
    when cancellation = 'null' or TRIM(cancellation) =""then NULL
    else cancellation
end as cancellation

from runner_orders;


"""
cursor.execute(query)

### Question and Answer

#### Pizza Metrics

How many pizzas were ordered?

In [5]:
query= """
select count(pizza_id) from c_view;
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(14,)


How many unique customer orders were made?

In [6]:
query= """
select count(distinct order_id) from c_view;

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(10,)


How many successful orders were delivered by each runner?

In [7]:
query= """
select runner_id ,count(*)
from r_view
where  cancellation is null
group by runner_id
;
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(1, 4)
(2, 3)
(3, 1)


How many of each type of pizza was delivered?

In [8]:
query= """
select c.pizza_id, count(*)
from r_view r inner join c_view c
on c.order_id = r.order_id
where  cancellation is null
group by c.pizza_id
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(1, 9)
(2, 3)


How many Vegetarian and Meatlovers were ordered by each customer?

In [9]:
query= """
select c.customer_id, p.pizza_name,count(*)
from c_view c inner join pizza_names p 
on c.pizza_id = p.pizza_id
group by c.customer_id, p.pizza_name
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(101, 'Meatlovers', 2)
(101, 'Vegetarian', 1)
(102, 'Meatlovers', 2)
(102, 'Vegetarian', 1)
(103, 'Meatlovers', 3)
(103, 'Vegetarian', 1)
(104, 'Meatlovers', 3)
(105, 'Vegetarian', 1)


What was the maximum number of pizzas delivered in a single order?

In [10]:
query= """
select max(pizza_number)
from (select order_id,count(pizza_id) as pizza_number
from c_view
group by order_id)
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(3,)


For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [11]:
query= """
select customer_id, 
sum(case
    when exclusions is not null or extras is not null then 1
    else 0
end) as no_of_changed_pizzas,
sum(case
when exclusions is not null or extras is not null then 0
    else 1
end) as no_of_unchanged_pizzas

from c_view c inner join r_view r
on c.order_id= r.order_id
where cancellation is null
group by customer_id 

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(101, 0, 2)
(102, 0, 3)
(103, 3, 0)
(104, 2, 1)
(105, 1, 0)


How many pizzas were delivered that had both exclusions and extras?

In [12]:
query= """
select count(*)
from c_view c inner join r_view r
on c.order_id = r.order_id
where r.cancellation is null and (c.exclusions is not null and c.extras is not null)
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(1,)


What was the total volume of pizzas ordered for each hour of the day?

In [14]:
query= """
select inner.hour, count(*) 
from(
    select strftime('%H', order_time) as hour
    from c_view    
) as inner
group by inner.hour

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('11', 1)
('13', 3)
('18', 3)
('19', 1)
('21', 3)
('23', 3)


What was the volume of orders for each day of the week?

In [15]:
query= """
select inner.day_of_week_name, count(*) 
from(
    select
    strftime('%w', order_time) AS day_of_week_numeric, 
    CASE strftime('%w', order_time)
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        WHEN '6' THEN 'Saturday'
        ELSE 'Unknown'
    END AS day_of_week_name
    from c_view   
    order by day_of_week_numeric asc
) as inner
group by inner.day_of_week_name
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('Friday', 1)
('Saturday', 5)
('Thursday', 3)
('Wednesday', 5)


## Runner and Customer Experience

How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [35]:
query= """
select 
cast ((julianday(registration_date) - julianday("2021-01-01"))/7 as integer) as week_of_registraion,count(runner_id) as number_of_registration
from runners
group by week_of_registraion
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(0, 2)
(1, 1)
(2, 1)


Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [39]:
query= """
with cte as(select c.order_id,count(pizza_id) as number_of_pizza, round((julianday(min(r.pickup_time))-julianday(min(c.order_time)))*24*60,2) as time_duration_in_minute
from c_view c inner join r_view r on 
    c.order_id = r.order_id
where r.pickup_time is not null
group by c.order_id)

select number_of_pizza, avg(time_duration_in_minute)
from cte 
group by number_of_pizza;
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

(1, 12.356)
(2, 18.375)
(3, 29.28)


In [41]:
if connection:
    connection.close()